In [19]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [20]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [21]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [22]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [23]:
from expiries import dict_expiries

In [24]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [25]:
bnf_1min = pd.read_csv("../data/nifty_min (2).csv")

In [26]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.tail()

,index,datetime,o,h,l,c,v
777613,nifty,2025-05-30 15:25:00,24741.4000,24742.7000,24740.5000,24741.7000,0
777614,nifty,2025-05-30 15:26:00,24742.2500,24746.2000,24740.3000,24740.3000,0
777615,nifty,2025-05-30 15:27:00,24741.0500,24749.0500,24739.5000,24747.1500,0
777616,nifty,2025-05-30 15:28:00,24746.5500,24746.8000,24731.1000,24745.2500,0
777617,nifty,2025-05-30 15:29:00,24743.6000,24749.3000,24731.8500,24736.6500,0


In [27]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [28]:
dict_expiries

{'nifty': [datetime.datetime(2017, 1, 25, 0, 0),
  datetime.datetime(2017, 2, 23, 0, 0),
  datetime.datetime(2017, 3, 30, 0, 0),
  datetime.datetime(2017, 4, 27, 0, 0),
  datetime.datetime(2017, 5, 25, 0, 0),
  datetime.datetime(2017, 6, 29, 0, 0),
  datetime.datetime(2017, 7, 27, 0, 0),
  datetime.datetime(2017, 8, 31, 0, 0),
  datetime.datetime(2017, 9, 28, 0, 0),
  datetime.datetime(2017, 10, 26, 0, 0),
  datetime.datetime(2017, 11, 30, 0, 0),
  datetime.datetime(2017, 12, 28, 0, 0),
  datetime.datetime(2018, 1, 25, 0, 0),
  datetime.datetime(2018, 2, 22, 0, 0),
  datetime.datetime(2018, 3, 28, 0, 0),
  datetime.datetime(2018, 4, 26, 0, 0),
  datetime.datetime(2018, 5, 31, 0, 0),
  datetime.datetime(2018, 6, 28, 0, 0),
  datetime.datetime(2018, 7, 26, 0, 0),
  datetime.datetime(2018, 8, 30, 0, 0),
  datetime.datetime(2018, 9, 27, 0, 0),
  datetime.datetime(2018, 10, 25, 0, 0),
  datetime.datetime(2018, 11, 29, 0, 0),
  datetime.datetime(2018, 12, 27, 0, 0),
  datetime.datetime(2019,

In [29]:
from datetime import date
from bisect import bisect_right

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_right(expiry_dates, input_date.date())
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [30]:
index_ = 'nifty'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8

In [31]:
import pandas as pd
import numpy as np

def calculate_weekly_levels(df, x_percent):
    # Convert datetime to Pandas datetime type
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Set datetime as index
    df = df.set_index('datetime')

    # Define week from Friday to Thursday
    df['Week'] = (df.index - pd.DateOffset(days=1)).to_period('W-THU')

    # Resample to get weekly OHLC (Friday Open - Thursday Close)
    weekly_df = df.resample('W-THU').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'})

    # Calculate weekly range and previous week's range
    weekly_df['Range'] = weekly_df['high'] - weekly_df['low']
    weekly_df['Prev_Week_Range'] = weekly_df['Range'].shift(1)

    return weekly_df

async def execute_trading_strategy(df, x_percent, tf, offset, entry_time, re_entry_threshold):
    
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'})
    weekly_levels = calculate_weekly_levels(df, x_percent)
    weekly_levels = weekly_levels[:-1]
    # print(weekly_levels.head().to_string())
    tradebook = []

    re_entry_num = 0

    # Iterate over all weeks
    for week_till, row in weekly_levels.iterrows():
        # print("Previous Week Till : ", week_till)
        if pd.isna(row['Prev_Week_Range']):  
            continue  # Skip the first week as we need the previous week's range

        # Get current week's Friday open
        current_friday_date = week_till + dt.timedelta(days=1)
        print("Current Friday : ", current_friday_date.date())
        current_friday_open = df.loc[df['datetime'].dt.date >= current_friday_date.date(), 'open'].iloc[0]

        # Dynamic multiplier
        # multiplier = 1 * np.sqrt(row['Range'] / current_friday_open)
        multiplier = x_percent * (row['Range'] / current_friday_open)
        if multiplier*100 > 10*x_percent:            # To avoid any trade with previous week range > 10% of current market open
            continue
        # print(multiplier)
        # Calculate high & low levels based on previous week's range
        # high_level = current_friday_open + (x_percent / 100) * row['Range']
        # low_level = current_friday_open - (x_percent / 100) * row['Range']

        high_level = current_friday_open + multiplier * row['Range']      # HL = 23000 + (0.21 * 500) = 23105
        low_level = current_friday_open - multiplier * row['Range']

        # Round levels to nearest 50
        high_level_strike = np.round(high_level / STRIKE_SPREAD_) * STRIKE_SPREAD_
        low_level_strike = np.round(low_level / STRIKE_SPREAD_) * STRIKE_SPREAD_

        # print(current_friday_open, high_level, high_level_strike, low_level, low_level_strike)

        # Initialize trade activity flags
        pe_trade_active = True
        ce_trade_active = True

        current_week_start = current_friday_date
        current_week_end = current_friday_date + dt.timedelta(days=6, hours=15, minutes=30)
        
        # Filter the DataFrame for the current week's data
        current_week_data = df[(df['datetime'] >= current_week_start) & (df['datetime'] <= current_week_end)]
        # print(current_week_data.to_string())

        expiry = get_next_expiry(current_week_start, index_)
        dte = (expiry - current_week_start.date()).days
        if dte >= 7:
            continue
        ce_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(high_level_strike),
            asset_class='C',
            start_date=current_week_start.date(),
            start_time=entry_time,
            end_date=expiry,
            end_time=dt.time(15, 30),
        )
        if not isinstance(ce_df, str) and ce_df is not None:
            ce_df = resample(ce_df, tf, offset)
            ce_df_pandas = ce_df.to_pandas()
            ce_entry_price = ce_df_pandas.iloc[0]['o']
        else:
            ce_entry_price = float('nan')

        pe_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(low_level_strike),
            asset_class='P',
            start_date=current_week_start.date(),
            start_time=entry_time,
            end_date=expiry,
            end_time=dt.time(15, 30),
        )
        if not isinstance(pe_df, str) and pe_df is not None:
            pe_df = resample(pe_df, tf, offset)
            pe_df_pandas = pe_df.to_pandas()
            pe_entry_price = pe_df_pandas.iloc[0]['o']
        else:
            pe_entry_price = float('nan')
        
        tradebook.append({
            'week_start': current_week_start,
            'entry_time': dt.datetime.combine(current_week_start, entry_time),
            'current week open': current_friday_open,
            'previous week high': row['high'],
            'previous week low': row['low'],
            'previous week range': row['Range'],
            'multiplier%': multiplier*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(high_level_strike),
            'type': 'CE',
            'expiry': expiry,
            "tag": "STRANGLE",
            # 'action': 'SELL'
            'entry price': ce_entry_price,
        })
        tradebook.append({
            'week_start': current_week_start,
            'entry_time': dt.datetime.combine(current_week_start, entry_time),
            'current week open': current_friday_open,
            'previous week high': row['high'],
            'previous week low': row['low'],
            'previous week range': row['Range'],
            'multiplier%': multiplier*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(low_level_strike),
            'type': 'PE',
            'expiry': expiry,
            "tag": "STRANGLE",
            # 'action': 'SELL'
            'entry price': pe_entry_price,
        })
        # print('CE Entry', ce_entry_price, '\nPE Entry', pe_entry_price)

        # Manage trades during the week
        # print(current_week_data.to_string())
        for i in range(0, len(current_week_data)):
            dte = (expiry - (current_week_data['datetime'].iloc[i]).date()).days
            # prev_close = current_week_data['close'].iloc[i - 1]
            curr_close = current_week_data['close'].iloc[i]
            curr_datetime = current_week_data['datetime'].iloc[i]
            # print(curr_datetime, curr_close, high_level, low_level)
            # Square off if breached
            if ce_trade_active and curr_close >= high_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'CE' and 
                        trade['week_start'] == current_week_start and 
                        'exit_time' not in trade
                    ):
                        df_row = ce_df_pandas[ce_df_pandas['datetime'] <= curr_datetime]
                        ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = ce_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('CE Exit', ce_exit_price)
                        break
                ce_trade_active = False  # Mark CE trade as inactive
        
            if pe_trade_active and curr_close <= low_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'PE' and
                        trade['week_start'] == current_week_start and 
                        'exit_time' not in trade
                    ):
                        df_row = pe_df_pandas[pe_df_pandas['datetime'] <= curr_datetime]
                        pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = pe_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('PE Exit', pe_exit_price)
                        break
                        
                pe_trade_active = False  # Mark PE trade as inactive
        
            # Re-enter if closed back inside range
            if not ce_trade_active and curr_close <= high_level and dte >= 0 and re_entry_num < re_entry_threshold:
                df_row = ce_df_pandas[ce_df_pandas['datetime'] >= curr_datetime]
                ce_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('CE Entry', ce_entry_price)
                tradebook.append(
                    {
                        'week_start': current_week_start,
                        'entry_time': curr_datetime,
                        'current week open': current_friday_open,
                        'previous week high': row['high'],
                        'previous week low': row['low'],
                        'previous week range': row['Range'],
                        'multiplier%': multiplier*100,
                        'high level': high_level,
                        'low level': low_level,
                        'strike': int(high_level_strike),
                        'type': 'CE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        # 'action': 'SELL'
                        'entry price': ce_entry_price,
                    }
                )
                ce_trade_active = True  # Reactivate CE trade
                re_entry_num += 1
        
            if not pe_trade_active and curr_close >= low_level and dte >= 0 and re_entry_num < re_entry_threshold:
                df_row = pe_df_pandas[pe_df_pandas['datetime'] >= curr_datetime]
                pe_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('PE Entry', pe_entry_price)
                tradebook.append(
                    {
                        'week_start': current_week_start,
                        'entry_time': curr_datetime,
                        'current week open': current_friday_open,
                        'previous week high': row['high'],
                        'previous week low': row['low'],
                        'previous week range': row['Range'],
                        'multiplier%': multiplier*100,
                        'high level': high_level,
                        'low level': low_level,
                        'strike': int(low_level_strike),
                        'type': 'PE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        # 'action': 'SELL'
                        'entry price': pe_entry_price,
                    }
                )
                pe_trade_active = True  # Reactivate PE trade
                re_entry_num += 1

            if curr_datetime == current_week_data.iloc[-1]['datetime']:
                re_entry_num = 0
                for trade in tradebook:
                    if (
                        trade['week_start'] == current_week_start and
                        'exit_time' not in trade
                    ):
                        trade['exit_time'] = curr_datetime
                        
                        if trade['type'] == 'CE':
                            trade['exit price'] = ce_df_pandas['c'].iloc[-1]
                            # print('CE Exit', ce_df_pandas['c'].iloc[-1])
                        else:
                            trade['exit price'] = pe_df_pandas['c'].iloc[-1]
                            # print('PE Exit', pe_df_pandas['c'].iloc[-1])

    tb = pd.DataFrame(tradebook)
    tb['entry_time'] = pd.to_datetime(tb['entry_time'])
    tb['expiry'] = pd.to_datetime(tb['expiry'])
    tb['dte'] = (tb['expiry'].dt.date - tb['entry_time'].dt.date).apply(lambda x: x.days)
    tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
    tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
    tb['portfolio'] = 20_00_000
    tb['index leverage'] = np.where(tb['type'] == 'PE', 9, 10)
    tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike'] 
    tb['PnL'] = tb['qty'] * tb['final points']
    tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
    tb['Trade Year'] = tb['week_start'].dt.year
    
    return tb


In [32]:
tf = '10m'
offset = '5m'
bnf_1h = resample(pl.DataFrame(bnf_1min), tf, offset)
tb = await execute_trading_strategy(bnf_1h.to_pandas(), 5, tf, offset, dt.time(9, 16), 4)

Current Friday :  2019-01-11
Current Friday :  2019-01-18
Current Friday :  2019-01-25
Current Friday :  2019-02-01
Current Friday :  2019-02-08
Current Friday :  2019-02-15
Current Friday :  2019-02-22
Current Friday :  2019-03-01
Current Friday :  2019-03-08
Current Friday :  2019-03-15
Current Friday :  2019-03-22
Current Friday :  2019-03-29
Current Friday :  2019-04-05
Current Friday :  2019-04-12
Current Friday :  2019-04-19
Current Friday :  2019-04-26
Current Friday :  2019-05-03
Current Friday :  2019-05-10
Current Friday :  2019-05-17
Current Friday :  2019-05-24
Current Friday :  2019-05-31
Current Friday :  2019-06-07
Current Friday :  2019-06-14
Current Friday :  2019-06-21
Current Friday :  2019-06-28
Current Friday :  2019-07-05
Current Friday :  2019-07-12
Current Friday :  2019-07-19
Current Friday :  2019-07-26
Current Friday :  2019-08-02
Current Friday :  2019-08-09
Current Friday :  2019-08-16
Current Friday :  2019-08-23
Current Friday :  2019-08-30
Current Friday

In [33]:
tb.to_csv('rbws_nifty.csv', index=False)

In [34]:
# tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
# tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
# tb['portfolio'] = 1_00_00_000
# tb['index leverage'] = 8
# tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike']
# tb['PnL'] = tb['qty'] * tb['final points']
# tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
# tb['Trade Year'] = tb['week_start'].dt.year

In [18]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [19]:
tb_may = tb[(tb['Trade Year'] == 2025) & (tb['entry_time'].dt.month == 5)]

In [35]:
tb.head(30)

,week_start,entry_time,current week open,previous week high,previous week low,previous week range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit price,exit_time,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year
0,2019-01-25,2019-01-25 09:16:00,10859.7500,10987.2000,10798.9000,188.3000,8.6696,10876.0749,10843.4251,10900,CE,2019-01-31,STRANGLE,93.9000,100.5000,2019-01-25 09:15:00,6,1.9440,-8.5440,2000000,10,1834.8624,-15677.0642,-0.7839,2019
1,2019-01-25,2019-01-25 09:16:00,10859.7500,10987.2000,10798.9000,188.3000,8.6696,10876.0749,10843.4251,10850,PE,2019-01-31,STRANGLE,46.0000,86.2500,2019-01-25 14:05:00,6,1.3225,-41.5725,2000000,9,1658.9862,-68968.2028,-3.4484,2019
2,2019-01-25,2019-01-25 13:35:00,10859.7500,10987.2000,10798.9000,188.3000,8.6696,10876.0749,10843.4251,10900,CE,2019-01-31,RE-ENTRY,68.3000,0.0500,2019-01-31 15:25:00,6,0.6835,67.5665,2000000,10,1834.8624,123975.2294,6.1988,2019
3,2019-02-08,2019-02-08 09:16:00,11023.5000,11117.9000,10813.6500,304.2500,13.8001,11065.4867,10981.5133,11050,CE,2019-02-14,STRANGLE,39.8000,0.0500,2019-02-14 15:25:00,6,0.3985,39.3515,2000000,10,1809.9548,71224.4344,3.5612,2019
4,2019-02-08,2019-02-08 09:16:00,11023.5000,11117.9000,10813.6500,304.2500,13.8001,11065.4867,10981.5133,11000,PE,2019-02-14,STRANGLE,113.6000,NaN,2019-02-08 12:55:00,6,NaN,NaN,2000000,9,1636.3636,NaN,NaN,2019
5,2019-02-08,2019-02-08 13:35:00,11023.5000,11117.9000,10813.6500,304.2500,13.8001,11065.4867,10981.5133,11000,PE,2019-02-14,RE-ENTRY,126.1500,NaN,2019-02-08 14:55:00,6,NaN,NaN,2000000,9,1636.3636,NaN,NaN,2019
6,2019-02-15,2019-02-15 09:16:00,10780.2500,11041.1500,10718.8500,322.3000,14.9486,10828.4294,10732.0706,10850,CE,2019-02-21,STRANGLE,34.1500,0.0500,2019-02-21 15:25:00,6,0.3420,33.7580,2000000,10,1843.3180,62226.7281,3.1113,2019
7,2019-02-15,2019-02-15 09:16:00,10780.2500,11041.1500,10718.8500,322.3000,14.9486,10828.4294,10732.0706,10750,PE,2019-02-21,STRANGLE,86.0000,92.0000,2019-02-15 09:15:00,6,1.7800,-7.7800,2000000,9,1674.4186,-13026.9767,-0.6513,2019
8,2019-02-15,2019-02-20 15:05:00,10780.2500,11041.1500,10718.8500,322.3000,14.9486,10828.4294,10732.0706,10750,PE,2019-02-21,RE-ENTRY,39.4500,34.7500,2019-02-21 09:45:00,1,0.7420,3.9580,2000000,9,1674.4186,6627.3488,0.3314,2019
9,2019-02-15,2019-02-21 09:55:00,10780.2500,11041.1500,10718.8500,322.3000,14.9486,10828.4294,10732.0706,10750,PE,2019-02-21,RE-ENTRY,31.1000,32.8000,2019-02-21 10:05:00,0,0.6390,-2.3390,2000000,9,1674.4186,-3916.4651,-0.1958,2019


# 9:16 variation below

In [21]:
stats = generate_stats(tb, 'RBWS')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,71.4557,209,33.9713,3.1081,-1.0972,-16.6572,4.2898,RBWS
2020,107.6755,212,34.9057,4.5404,-1.6545,-19.2987,5.5794,RBWS
2021,106.5985,238,36.1345,3.5842,-1.3354,-20.4305,5.2176,RBWS
2022,68.8231,250,30.0000,4.1142,-1.3858,-19.5717,3.5164,RBWS
2023,65.0748,247,29.5547,2.6721,-0.7784,-7.1444,9.1084,RBWS
2024,36.3286,228,32.4561,3.0818,-1.2449,-20.8296,1.7441,RBWS
2025,19.9803,94,31.9149,3.4298,-1.2955,-11.8495,1.6862,RBWS
Overall,475.9364,1478,32.6793,3.5186,-1.2447,-20.8296,22.8490,RBWS


In [76]:
stats = generate_stats(tb, 'RBWS')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,58.4469,211,34.5972,3.0308,-1.1971,-20.6278,2.8334,RBWS
2020,114.4488,211,35.0711,4.5601,-1.6278,-19.2987,5.9304,RBWS
2021,108.0251,234,35.8974,3.6641,-1.3407,-20.4305,5.2875,RBWS
2022,66.4706,249,30.1205,4.1142,-1.4075,-19.5717,3.3963,RBWS
2023,65.0748,247,29.5547,2.6721,-0.7784,-7.1444,9.1084,RBWS
2024,32.8506,228,31.1404,3.1737,-1.2580,-20.8296,1.5771,RBWS
2025,10.8471,61,16.3934,4.7979,-1.0921,-11.8495,0.9154,RBWS
Overall,456.1639,1441,31.9223,3.5727,-1.2524,-20.8296,21.8997,RBWS


In [199]:
tb_ce = tb[tb['type'] == 'CE']
tb_pe = tb[tb['type'] == 'PE']
tb_strangle = tb[tb['tag'] == 'STRANGLE']

In [200]:
stats = generate_stats(tb_strangle, 'STR RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,31.9050,96,40.6250,3.9065,-2.1508,-23.0131,1.3864,STR RBW w Strangle
2020,113.9340,94,47.8723,5.6955,-2.9054,-17.2923,6.5887,STR RBW w Strangle
2021,43.2558,106,38.6792,4.7414,-2.3616,-20.5874,2.1011,STR RBW w Strangle
2022,55.2629,104,39.4231,5.4446,-2.7536,-22.6641,2.4383,STR RBW w Strangle
2023,19.0451,104,32.6923,3.0573,-1.2672,-16.2513,1.1719,STR RBW w Strangle
2024,1.7440,104,34.6154,3.8907,-2.0958,-44.8314,0.0389,STR RBW w Strangle
2025,22.9926,18,38.8889,5.8956,-1.6615,-10.3084,2.2305,STR RBW w Strangle
Overall,288.1395,626,38.8179,4.5743,-2.2017,-44.8314,6.4272,STR RBW w Strangle


In [201]:
tb_ce_str = tb_strangle[tb_strangle['type'] == 'CE']
tb_pe_str = tb_strangle[tb_strangle['type'] == 'PE']

In [202]:
stats = generate_stats(tb_ce_str, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,-16.2887,48,37.5000,3.6467,-2.7310,-28.2769,-0.5760,RBW w Strangle
2020,36.5315,47,40.4255,6.1909,-2.8963,-34.6825,1.0533,RBW w Strangle
2021,42.1767,53,43.3962,4.3513,-1.9301,-9.8132,4.2980,RBW w Strangle
2022,43.2957,52,48.0769,5.0955,-3.2343,-15.6392,2.7684,RBW w Strangle
2023,9.7318,52,32.6923,2.9603,-1.2301,-9.1993,1.0579,RBW w Strangle
2024,-2.1216,52,32.6923,4.1305,-2.1277,-26.0057,-0.0816,RBW w Strangle
2025,24.3247,9,55.5556,5.8821,-1.2714,-4.5154,5.3871,RBW w Strangle
Overall,137.6501,313,39.6166,4.5217,-2.2867,-34.6825,3.9689,RBW w Strangle


In [203]:
stats = generate_stats(tb_pe_str, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,48.1937,48,43.7500,4.1291,-1.4814,-13.2609,3.6343,RBW w Strangle
2020,77.4025,47,55.3191,5.3335,-2.9176,-30.5575,2.5330,RBW w Strangle
2021,1.0791,53,33.9623,5.2398,-2.7423,-30.7183,0.0351,RBW w Strangle
2022,11.9672,52,30.7692,5.9901,-2.3964,-25.1195,0.4764,RBW w Strangle
2023,9.3133,52,32.6923,3.1543,-1.3033,-11.0359,0.8439,RBW w Strangle
2024,3.8656,52,36.5385,3.6761,-2.0619,-30.4898,0.1268,RBW w Strangle
2025,-1.3321,9,22.2222,5.9293,-1.8844,-5.7930,-0.2300,RBW w Strangle
Overall,150.4894,313,38.0192,4.6291,-2.1184,-34.0883,4.4147,RBW w Strangle


# TF : 10min , Multiplier : 5 * (prev. week range / current open)
# No trades if previous week range > 10% of current spot price

In [92]:
tb.to_csv('rbws_nifty.csv', index=False)

In [63]:
roi_summary = tb.groupby('tag').agg(
    total_roi_pct=('ROI%', 'sum'),
    trade_count=('ROI%', 'count'),
    win_count=('ROI%', lambda x: (x > 0).sum())
).reset_index()

# Calculate win rate as a percentage
roi_summary['win_rate_pct'] = (roi_summary['win_count'] / roi_summary['trade_count']) * 100

# Optional: drop win_count column if not needed
roi_summary = roi_summary.drop(columns='win_count')
roi_summary

,tag,total_roi_pct,trade_count,win_rate_pct
0,RE-ENTRY,211.6963,1202,27.0383
1,STRANGLE,288.1395,617,39.3841


In [467]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,67.1717,284,33.8028,2.3866,-0.8707,-17.0875,3.9311,RBW w Strangle
2020,91.3731,267,31.8352,3.8527,-1.2973,-21.0607,4.3386,RBW w Strangle
2021,100.6335,297,36.0269,2.8830,-1.0997,-20.0095,5.0293,RBW w Strangle
2022,81.9807,315,30.4762,3.3011,-1.0826,-13.7765,5.9508,RBW w Strangle
2023,65.5546,312,31.0897,1.9529,-0.5984,-7.2697,9.0175,RBW w Strangle
2024,47.8182,287,35.5401,2.2067,-0.9634,-16.8124,2.8442,RBW w Strangle
2025,-0.6254,100,26.0000,2.6235,-0.9302,-17.3922,-0.0360,RBW w Strangle
Overall,453.9064,1862,32.7068,2.7335,-0.9772,-21.0607,21.5523,RBW w Strangle


In [468]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,67.1717,284,33.8028,2.3866,-0.8707,-17.0875,3.9311,RBW w Strangle
2020,91.3731,267,31.8352,3.8527,-1.2973,-21.0607,4.3386,RBW w Strangle
2021,100.6335,297,36.0269,2.8830,-1.0997,-20.0095,5.0293,RBW w Strangle
2022,81.9807,315,30.4762,3.3011,-1.0826,-13.7765,5.9508,RBW w Strangle
2023,65.5546,312,31.0897,1.9529,-0.5984,-7.2697,9.0175,RBW w Strangle
2024,47.8182,287,35.5401,2.2067,-0.9634,-16.8124,2.8442,RBW w Strangle
2025,-0.6254,100,26.0000,2.6235,-0.9302,-17.3922,-0.0360,RBW w Strangle
Overall,453.9064,1862,32.7068,2.7335,-0.9772,-21.0607,21.5523,RBW w Strangle


In [470]:
tb.to_csv("RBW w Strangle 10min.csv", index=False)

In [96]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,41.4154,132,64.3939,1.5856,-2.0296,-18.2533,2.2689,RBW w Strangle
2020,40.2254,145,62.7586,2.9047,-4.1501,-61.4071,0.6551,RBW w Strangle
2021,46.4850,145,62.7586,1.7585,-2.1835,-18.3801,2.5291,RBW w Strangle
2022,44.4147,139,58.9928,2.1531,-2.4471,-15.9400,2.7864,RBW w Strangle
2023,30.9280,137,62.7737,1.1154,-1.3265,-7.8012,3.9645,RBW w Strangle
2024,22.7406,129,62.7907,1.5129,-2.0792,-13.9160,1.6341,RBW w Strangle
2025,8.9918,37,62.1622,1.5597,-1.9200,-7.6133,1.1811,RBW w Strangle
Overall,235.2010,864,62.3843,1.8368,-2.3812,-65.1730,3.6089,RBW w Strangle


# Original Stats with 50% Range Multiplier and 1hr TF

In [89]:
weekly_roi = tb.groupby('week_start')['ROI%'].sum().reset_index()
weekly_roi['Trade Year'] = weekly_roi['week_start'].dt.year

In [90]:
weekly_roi

,week_start,ROI%,Trade Year
0,2019-01-25,1.9665,2019
1,2019-02-08,3.5612,2019
2,2019-02-15,4.8457,2019
3,2019-02-22,1.0523,2019
4,2019-03-01,5.8110,2019
5,2019-03-08,2.5861,2019
6,2019-03-15,-4.8674,2019
7,2019-03-22,5.2782,2019
8,2019-03-29,1.7191,2019
9,2019-04-05,-2.8489,2019


In [91]:
stats = generate_stats(weekly_roi, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,58.4469,48,70.8333,2.8707,-2.7969,-11.6398,5.0213,RBW w Strangle
2020,114.4488,47,74.4681,4.6314,-3.9710,-10.8523,10.5461,RBW w Strangle
2021,108.0251,53,79.2453,3.3558,-2.9926,-10.8511,9.9552,RBW w Strangle
2022,66.4706,52,65.3846,3.4826,-3.0551,-9.2644,7.1748,RBW w Strangle
2023,65.0748,52,78.8462,1.9986,-1.6870,-3.3144,19.6340,RBW w Strangle
2024,32.8506,52,67.3077,2.5458,-3.3089,-16.7365,1.9628,RBW w Strangle
2025,10.8471,13,38.4615,3.5500,-1.7258,-5.4772,1.9804,RBW w Strangle
Overall,456.1639,317,71.2934,3.1321,-2.9610,-16.7365,27.2556,RBW w Strangle


# Weekly Stats

In [188]:
# SIMULATION
# multiplier_range = [1, 2, 3, 4, 5]
# tf = ['5m', '10m', '15m', '20m', '30m', '45m', '60m']
tf = ['10m']
multiplier_range = [5]
entry_times = [dt.time(13, 30), dt.time(13, 45), dt.time(14, 0), dt.time(14, 20), dt.time(14, 35), dt.time(14, 50)]
for i in multiplier_range:
    for j in tf:
        for entry_time in entry_times:
            variation = f'Multiplier {i}, TF {j}, Entry Time : {entry_time}'
            print(variation)
            
            if j == '10m':
                offset = '5m'
            elif j >= '20m':
                offset = '15m'
            else:
                offset = '0m'
                
            bnf_1h = resample(pl.DataFrame(bnf_1min), j, offset)
            tb = await execute_trading_strategy(bnf_1h.to_pandas(), i, j, offset, entry_time)
    
            if len(tb) > 0:
                stats = generate_stats(tb, variation)
                for x, y in stats.items():
                    z = pd.DataFrame(y)
                    print(z.to_string())


Multiplier 5, TF 10m, Entry Time : 13:30:00
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                                    Variation
2019      88.0806          284  30.9859                2.7013             -1.1167     -19.2138       4.5842  Multiplier 5, TF 10m, Entry Time : 13:30:00
2020     100.0590          267  28.8390                4.6125             -1.7121     -16.7412       5.9768  Multiplier 5, TF 10m, Entry Time : 13:30:00
2021     111.7291          297  31.3131                3.4448             -1.4797     -24.5314       4.5545  Multiplier 5, TF 10m, Entry Time : 13:30:00
2022      36.7541          315  25.7143                4.1068             -1.5908     -22.2079       1.6550  Multiplier 5, TF 10m, Entry Time : 13:30:00
2023      62.1702          312  24.0385                2.4778             -0.8030     -12.4451       4.9956  Multiplier 5, TF 10m, Entry Time : 13:30:00
2024      50.5117          287  28.919

In [20]:
tb.tail()

,week_start,entry_time,current week open,previous week high,previous week low,previous week range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit price,exit_time,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year
1831,2025-02-21,2025-02-21 09:16:00,22855.8000,23133.7000,22725.4500,408.2500,8.9310,22892.2608,22819.3392,22800,PE,2025-02-27,STRANGLE,112.0000,135.4000,2025-02-21 09:45:00,6,2.4740,-25.8740,10000000,10,4385.9649,-113482.4561,-1.1348,2025
1832,2025-02-21,2025-02-21 09:25:00,22855.8000,23133.7000,22725.4500,408.2500,8.9310,22892.2608,22819.3392,22900,CE,2025-02-27,RE-ENTRY,162.9500,0.1000,2025-02-27 15:25:00,6,1.6305,161.2195,10000000,11,4803.4934,774416.8122,7.7442,2025
1833,2025-02-21,2025-02-21 12:25:00,22855.8000,23133.7000,22725.4500,408.2500,8.9310,22892.2608,22819.3392,22800,PE,2025-02-27,RE-ENTRY,122.2500,125.7500,2025-02-21 12:35:00,6,2.4800,-5.9800,10000000,10,4385.9649,-26228.0702,-0.2623,2025
1834,2025-02-28,2025-02-28 09:16:00,22433.4000,22921.0000,22508.4000,412.6000,9.1961,22471.3431,22395.4569,22450,CE,2025-03-06,STRANGLE,132.8000,42.2500,2025-02-28 15:25:00,6,1.7505,88.7995,10000000,11,4899.7773,435097.7728,4.3510,2025
1835,2025-02-28,2025-02-28 09:16:00,22433.4000,22921.0000,22508.4000,412.6000,9.1961,22471.3431,22395.4569,22400,PE,2025-03-06,STRANGLE,148.5000,138.9500,2025-02-28 09:15:00,6,2.8745,6.6755,10000000,10,4464.2857,29801.3393,0.2980,2025


In [23]:
def round_to_nearest_50(value):
    return 50 * round(value / 50)


async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 3
    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_to_nearest_50(row["strike"] * (1 + hedge_pct / 100))
            if row["type"] == "CE"
            else round_to_nearest_50(row["strike"] * (1 - hedge_pct / 100))
        ),
        axis=1,
    )

    # df["Hedge Contract"] = df.apply(
    #     lambda row: get_option_contract_name2(
    #         index_name, row["Hedge Strike"], row["expiry"], row["type"]
    #     ),
    #     axis=1,
    # )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    df['entry_time'] = pd.to_datetime(df['entry_time'])
    df['exit_time'] = pd.to_datetime(df['exit_time'])

    print(df.head().to_string())

    for i in range(0, len(df)):
        print(df.iloc[i]["week_start"])
        hedge_strike = df.iloc[i]['Hedge Strike']
        hedge_expiry = df.iloc[i]["expiry"]
        hedge_opt_type = df.iloc[i]["type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["entry_time"] + dt.timedelta(minutes=9)
        hedge_exit_datetime = df.iloc[i]["exit_time"] + dt.timedelta(minutes=10)

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=int(hedge_strike),
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge_tmp = await add_hedges(tb)
# tb_with_hedge

  week_start          entry_time  current week open  previous week high  previous week low  previous week range  multiplier%  high level  low level  strike type     expiry       tag  entry price  exit price           exit_time  dte  slippage  final points  portfolio  index leverage       qty          PnL    ROI%  Trade Year  Hedge Strike  Hedge Entry Price  Hedge Exit Price
0 2019-01-25 2019-01-25 09:16:00         10859.7500          10987.2000         10798.9000             188.3000       8.6696  10876.0749 10843.4251   10900   CE 2019-01-31  STRANGLE      93.9000    100.5000 2019-01-25 09:15:00    6    1.9440       -8.5440   10000000              10 9174.3119  -78385.3211 -0.7839        2019         11250                NaN               NaN
1 2019-01-25 2019-01-25 09:16:00         10859.7500          10987.2000         10798.9000             188.3000       8.6696  10876.0749 10843.4251   10850   PE 2019-01-31  STRANGLE      46.0000     86.2500 2019-01-25 14:05:00    6    1.3225     

In [24]:
tb_with_hedge = tb_with_hedge_tmp

In [25]:
tb_with_hedge["Qty"] = tb_with_hedge["qty"] * 1
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
# tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
# tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL"] + tb_with_hedge["Hedge PnL"]

In [26]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 10000000
tb_with_hedge["entry_time"] = pd.to_datetime(tb_with_hedge["entry_time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["entry_time"].dt.year

In [27]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,55.0589,207,33.3333,2.8179,-1.0248,-15.5097,3.5500
2020,62.9304,214,32.2430,3.7549,-1.3528,-17.9183,3.5121
2021,79.5867,238,34.4538,3.2177,-1.1888,-15.8696,5.0150
2022,39.5660,249,28.1124,3.7567,-1.2622,-20.5556,1.9248
2023,46.6963,244,28.6885,2.5191,-0.7763,-7.2108,6.4759
2024,32.7574,231,31.6017,2.8936,-1.1296,-22.2738,1.4707
Overall,316.5956,1383.0000,31.2585,3.1543,-1.1298,-22.2738,14.2138


In [28]:
tb_with_hedge.tail(49)

,week_start,entry_time,current week open,previous week high,previous week low,previous week range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit price,exit_time,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year,Hedge Strike,Hedge Entry Price,Hedge Exit Price,Qty,Hedge Points,Hedge PnL,Total PnL,Total ROI%
1429,2025-02-28,2025-03-06 11:45:00,22433.4000,22921.0000,22508.4000,412.6000,9.1961,22471.3431,22395.4569,22400,PE,2025-03-06,RE-ENTRY,38.2500,0.0500,2025-03-06 15:25:00,0,0.3830,37.8170,10000000,9,4017.8571,151943.3036,1.5194,2025,21750,0.1500,0.0500,4017.8571,-0.1000,-401.7857,151541.5179,1.5154
1430,2025-03-07,2025-03-07 09:16:00,22508.6500,22556.4500,21964.6000,591.8500,13.1472,22586.4615,22430.8385,22600,CE,2025-03-13,STRANGLE,110.1000,163.6000,2025-03-07 10:05:00,6,2.7370,-56.2370,10000000,10,4424.7788,-248836.2832,-2.4884,2025,23300,3.5500,4.1000,4424.7788,0.5500,2433.6283,-246402.6549,-2.4640
1431,2025-03-07,2025-03-07 09:16:00,22508.6500,22556.4500,21964.6000,591.8500,13.1472,22586.4615,22430.8385,22450,PE,2025-03-13,STRANGLE,130.0500,187.6000,2025-03-11 09:15:00,6,3.1765,-60.7265,10000000,9,4008.9087,-243446.9933,-2.4345,2025,21800,10.9000,7.1500,4008.9087,-3.7500,-15033.4076,-258480.4009,-2.5848
1432,2025-03-07,2025-03-07 10:25:00,22508.6500,22556.4500,21964.6000,591.8500,13.1472,22586.4615,22430.8385,22600,CE,2025-03-13,RE-ENTRY,152.8500,154.5000,2025-03-07 10:35:00,6,3.0735,-4.7235,10000000,10,4424.7788,-20900.4425,-0.2090,2025,23300,4.3000,4.4000,4424.7788,0.1000,442.4779,-20457.9646,-0.2046
1433,2025-03-07,2025-03-07 10:45:00,22508.6500,22556.4500,21964.6000,591.8500,13.1472,22586.4615,22430.8385,22600,CE,2025-03-13,RE-ENTRY,139.5000,154.4000,2025-03-07 10:55:00,6,2.9390,-17.8390,10000000,10,4424.7788,-78933.6283,-0.7893,2025,23300,3.9500,4.1000,4424.7788,0.1500,663.7168,-78269.9115,-0.7827
1434,2025-03-07,2025-03-07 11:45:00,22508.6500,22556.4500,21964.6000,591.8500,13.1472,22586.4615,22430.8385,22600,CE,2025-03-13,RE-ENTRY,134.4500,148.0000,2025-03-07 14:25:00,6,2.8245,-16.3745,10000000,10,4424.7788,-72453.5398,-0.7245,2025,23300,2.9500,2.6000,4424.7788,-0.3500,-1548.6726,-74002.2124,-0.7400
1435,2025-03-07,2025-03-07 14:45:00,22508.6500,22556.4500,21964.6000,591.8500,13.1472,22586.4615,22430.8385,22600,CE,2025-03-13,RE-ENTRY,142.0000,122.3500,2025-03-10 09:15:00,6,2.6435,17.0065,10000000,10,4424.7788,75250.0000,0.7525,2025,23300,2.5000,2.1000,4424.7788,-0.4000,-1769.9115,73480.0885,0.7348
1436,2025-03-14,2025-03-14 09:16:00,22353.1500,22676.7500,22314.7000,362.0500,8.0984,22382.4703,22323.8297,22400,CE,2025-03-20,STRANGLE,165.5000,222.2500,2025-03-17 09:15:00,6,3.8775,-60.6275,10000000,10,4464.2857,-270658.4821,-2.7066,2025,23050,5.8000,5.6500,4464.2857,-0.1500,-669.6429,-271328.1250,-2.7133
1437,2025-03-14,2025-03-14 09:16:00,22353.1500,22676.7500,22314.7000,362.0500,8.0984,22382.4703,22323.8297,22300,PE,2025-03-20,STRANGLE,89.9500,0.0500,2025-03-20 15:25:00,6,0.9000,89.0000,10000000,9,4035.8744,359192.8251,3.5919,2025,21650,3.2500,0.0500,4035.8744,-3.2000,-12914.7982,346278.0269,3.4628
1438,2025-03-21,2025-03-21 09:16:00,23168.2500,23216.7000,22353.1500,863.5500,18.6365,23329.1855,23007.3145,23350,CE,2025-03-27,STRANGLE,65.3000,150.7000,2025-03-21 10:45:00,6,2.1600,-87.5600,10000000,10,4282.6552,-374989.2934,-3.7499,2025,24050,2.5000,5.3500,4282.6552,2.8500,12205.5675,-362783.7259,-3.6278


In [29]:
tb_with_hedge.to_csv('rbws_nifty_w_hedge.csv', index=False)